##Setup

In [1]:
#connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install torch torchvision --upgrade

In [3]:
!pip install numpy -- U
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [4]:
!pip install -U transformers
#!pip install vllm==0.5.3.post1
# !pip install -U torch==2.5.1
!pip install pandas
# !pip install torch==2.0.1 torchvision==0.15.2



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 129.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0


In [5]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [6]:

#get data file
!wget -q -O data.csv "https://docs.google.com/uc?export=download&id=13NuKWse5ZLbE7-3WtmdzYSeOOaJ3MydG"


In [7]:
import pandas as pd
import torch

data = pd.read_csv("data.csv")
#print(data.head())
print(data.columns)

Index(['id_1', 'id_2', 'text_1', 'text_2', '1_better', 'prompt'], dtype='object')


In [8]:
df = data.sample(n=100, random_state=42)
print(df.columns)

Index(['id_1', 'id_2', 'text_1', 'text_2', '1_better', 'prompt'], dtype='object')


In [9]:
rubric = '''A high-scoring essay has native-like facility in the use of language with syntactic variety, appropriate word choice and phrases; well-controlled text organization; precise use of grammar and conventions; rare language inaccuracies that do not impede communication.
A low scoring essay has a limited range of familiar words or phrases loosely strung together; frequent errors in grammar (including syntax) and usage. Communication impeded in most cases by language inaccuracies.'''

rubric_cohesion = '''A high-scoring essay has text organization consistently well-controlled using a variety of effective linguistic features such as reference and transitional words and phrases to connect ideas across sentences and paragraphs; appropriate overlap of ideas.
A low scoring essay has no clear control of organization; cohesive devices not present or unsuccessfully used; presentation of ideas unclear'''

rubric_syntax = '''A high-scoring essay has flexible and effective use of a full range of syntactic structures including simple, compound, and complex sentences; there may be rare minor and negligible errors in sentence formation.
A low-scoring essay has pervasive and basic errors in sentence structure and word order that cause confusion; basic sentence errors are common.'''


rubric_vocab = '''A high-scoring essay has a wide range of vocabulary flexibly and effectively used to convey precise meanings; skillful use of topic-related terms and less common words; rare negligible inaccuracies in word use.
A low scoring essay has limited vocabulary often inappropriately used; limited control of word choice and word forms; little attempt to use topic-related terms.'''


rubric_phrase = '''A high scoring essay has flexible and effective use of a variety of phrases, such as idioms, collocations, and lexical bundles, to convey precise and subtle meanings; rare minor inaccuracies that are negligible.
A low scoring essay hsa memorized chunks of language, or simple phrasal patterns, predominate; many repetitions and misuses of phrases. '''


rubric_grammar = '''A high scoring essay has command of grammar and usage with few or no errors. A low scoring essay has errors in grammar and usage throughout.'''


rubric_conv = '''A high scoring essay has consistent use of appropriate conventions to convey meaning; spelling, capitalization, and punctuation errors nonexistent or negligible. A low scoring essay has minimal use of conventions; spelling, capitalization, and punctuation errors throughout.'''


In [10]:
from datasets import Dataset
from torch.utils.data import DataLoader
from torch import cuda, tensor, no_grad
import torch
from tqdm.auto import tqdm
import json


def preprocess(data):
  batch = {}
  batch['messages'] = [[{
      "role": "system",
      "content": f"You are an essay grader who COMPARES essays according to the rubric, and outputs ONLY the better essay: {rubric}"
  },
  {
      "role": "user",
      "content": f"Which of the following essays scores higher based on the rubric? Essay A: {essay_1}\n Essay B: {essay_2}\n Output which essay, A or B, scores HIGHER. The higher scoring essay is Essay "
  }] for essay_1, essay_2 in zip(data['text_1'], data['text_2'])]

  batch['id_1'] = data['id_1']
  batch['id_2'] = data['id_2']
  batch['prompt'] = data['prompt']
  return batch

def compute_metric():
    return

def collate(data):
    return {'messages': [ex['messages'] for ex in data],
            'id_1': [ex['id_1'] for ex in data],
            'id_2': [ex['id_2'] for ex in data],
            'prompt': [ex['prompt'] for ex in data]}

batch_size = 1
print('loading dataset...\n')
train_data = Dataset.from_pandas(df)
# train_data = train_data.select_columns(['full_text', 'text_id_kaggle'])
train_data = train_data.map(preprocess, batched=True, batch_size=batch_size)
train_dataloader = DataLoader(train_data,
                               batch_size=batch_size, shuffle=False,
                               collate_fn=collate)

loading dataset...



Map:   0%|          | 0/100 [00:00<?, ? examples/s]

##LLM grading

In [11]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.6 MB/s eta 0:00:00


In [12]:
import torch
import torchvision
print(f"PyTorch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")

PyTorch version: 2.6.0+cu124
torchvision version: 0.21.0+cu124


In [13]:
!pip install flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 107.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl size=187831595 sha256=58853b28a5a926cae14402bfd8d4d93a45ebf8f9e79533f37ab09d0d77a99c05
  Stored in directory: /root/.cache/pip/wheels/3d/88/d8/284b89f56af7d5bf366b10d6b8e251ac8a7c7bf3f04203fb4f
Successfully built flash-attn


In [14]:
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch import cuda, tensor, no_grad
import torch
from tqdm.auto import tqdm
import json
import accelerate

device = 'cuda' if cuda.is_available() else 'cpu'
pipe = pipeline(
    "text-generation",
    model="microsoft/Phi-4-mini-instruct",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    use_cache = False,
)

config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-4-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/54.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-4-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

merges.txt:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Device set to use cuda:0


In [15]:
epochs = 1
import csv

num_training_steps = epochs * len(train_dataloader)
progress_bar = tqdm(range(num_training_steps))
output_path = '/content/drive/MyDrive/senior_thesis/part_2/outputs/phi-4-mini-instruct.csv'

with open(output_path, 'w', newline='') as out_file:
    # Create a CSV writer that handles quoting properly
    csv_writer = csv.writer(out_file, quoting=csv.QUOTE_MINIMAL)

    # Write the header
    csv_writer.writerow(['id_1', 'id_2', 'prompt', 'pred'])

    with no_grad():
        for _ in range(epochs):
            for _, batch in enumerate(train_dataloader):
                id_1_values = batch['id_1']
                id_2_values = batch['id_2']
                prompt_values = batch['prompt']

                # Process the batch with the model
                out = pipe(batch['messages'], max_new_tokens=16, pad_token_id=pipe.tokenizer.eos_token_id)

                # Write each row properly
                for i in range(len(batch['messages'])):
                    # Handle id_2 properly (convert from list if necessary)
                    id_2 = id_2_values[i] if isinstance(id_2_values, list) else id_2_values

                    # Get the generated content
                    content = out[i][0]["generated_text"][2]["content"]

                    # Write to CSV properly (let the CSV writer handle quoting)
                    csv_writer.writerow([id_1_values[i], id_2, prompt_values[i], content])

                    # Print for debugging (optional)
                    print(f'{id_1_values[i]},{id_2},{prompt_values[i]},"{content}"')

                progress_bar.update()

# with open ('/content/drive/MyDrive/senior_thesis/part_2/outputs/phi-3-medium-4k-instruct.csv', 'w') as out_file:
#     out_file.write('id_1,id_2,prompt,pred\n')
#     with no_grad():
#         for _ in range(epochs):
#             for _, batch in enumerate(train_dataloader):
#                 id_1 = batch['id_1']
#                 id_2 = batch['id_2']
#                 prompt = batch['prompt']
#                 print(id_1)
#                 print(id_2)
#                 out = pipe(batch['messages'], max_new_tokens=16, pad_token_id=pipe.tokenizer.eos_token_id)
#                 # print(out[i][0]['generated_text'][2])
#                 for i in range(len(batch['messages'])):
#                     content = out[i][0]["generated_text"][2]["content"].replace('"', '""')

#                     out_file.write(f'{id_1[i]},{id_2},{prompt},{content}\n')
#                     print(f'{id_1[i]},{id_2[i]},{prompt[i]}"{content}"')
#                     #out_file.write(f'{essay_ids[i]},"{out[i][0]["generated_text"][2]["content"]}"\n')
#                     #print(f'{essay_ids[i]},"{out[i][0]["generated_text"][2]["content"]}"')
#                 progress_bar.update()

  0%|          | 0/100 [00:00<?, ?it/s]

A828480AA508,AA8DA5629CBC,Benefits of a good attitude,"Essay A scores higher based on the rubric. 

Essay A demonstrates a more sophisticated"
65B7FDE783F2,1732E7BCD28D,Four-day work week,"Essay B scores higher based on the rubric. 

Essay B demonstrates a more controlled"
7DC02C918305,98BD2D5840A1,Positive attitudes,"Essay B scores higher based on the rubric. 

Essay B demonstrates a more controlled"
6060D28C05B6,5AC16E188F1B,Distance learning,"Essay B scores higher based on the rubric. 

Essay B demonstrates a more controlled"
3F3DC123EDF6,CBFD5CD8BCD9,Curfews for teenagers,"Essay B scores higher based on the rubric. 

Essay B demonstrates a more controlled"
A8E686531564,9C9F45A47A92,Cell phones in classrooms,"Essay A scores higher based on the rubric. 

Essay A demonstrates a native-like"
E3668C4C4C0C,C2AECED57C95,Community service,"Essay B scores higher based on the rubric. 

Essay B demonstrates a more controlled"
04776677F17B,D49CB44F4364,Three-year high school program,"Essay A s

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


992DB55B9726,219E44AA7948,Learning from the experience of others,"Essay A scores higher based on the rubric. This essay demonstrates a native-like facility"
75C950F8EA18,13EDC505534E,Enjoyable educational activities,"Essay B scores higher based on the rubric. 

Essay B demonstrates a more controlled"
3EE4C10E9149,0C28481DFBB8,Being busy,"Essay A scores higher based on the rubric. 

Essay A demonstrates a native-like"
862ACD748291,1928846BB272,Career commitment,"Essay B scores higher based on the rubric. 

Essay B demonstrates a native-like"
FAB189F7E4DF,44FD30777ED7,Influencing behavior,"Essay B scores higher based on the rubric. 

Essay B demonstrates a native-like"
93E62B4919A6,A97ECE986D5F,Self-reliance,"Essay A scores higher based on the rubric. 

Essay A demonstrates a native-like"
E9705A9C0FE5,CE98C42F51C2,Career commitment,"Essay A scores higher based on the rubric. This essay demonstrates a native-like facility"
E4596B010B9B,9BF717F45D68,Mandatory extracurricular activities,"Es